In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf
from tensorflow.python.keras import backend as K

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
import time

# MSE Imports
import kepler_sieve
from asteroid_element import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts, random_elts, elts_add_mixture_params
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import load_ztf_batch, make_ztf_batch, ztf_score_by_elt, ztf_elt_summary
from asteroid_model import AsteroidPosition, AsteroidDirection, make_model_ast_pos
from asteroid_search_layers import CandidateElements, TrajectoryScore
from asteroid_search_model import AsteroidSearchModel, make_opt_adam
from asteroid_search_report import traj_diff
from candidate_element import score_by_elt
from astro_utils import deg2dist, dist2deg, dist2sec

Found 4 GPUs.  Setting memory growth = True.


In [2]:
# Aliases
keras = tf.keras

# Constants
dtype = tf.float32
dtype_np = np.float32
space_dims = 3

In [3]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

In [4]:
# Color for plots
color_mean = 'blue'
color_lo = 'orange'
color_hi = 'green'
color_min = 'red'
color_max = 'purple'

## Load ZTF Data and Batch of Orbital Elements

In [5]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [6]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [7]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top batch_size
idx = np.argsort(hit_count)[::-1]

# Extract the asteroid number and hit count for this batch
ast_num_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [8]:
# Parameters to build elements batch
batch_size = 64

# Batch of unperturbed elements
elts_ast = asteroid_elts(ast_nums=ast_num_best[0:batch_size])

In [9]:
# # Review unperturbed elements
# elts_ast

In [10]:
# Inputs to perturb elements: large
sigma_a = 0.05
sigma_e = 0.01
sigma_inc_deg = 0.25
sigma_f_deg = 1.0
sigma_Omega_deg = 1.0
sigma_omega_deg = 1.0
mask_pert = None
random_seed = 42

# Perturb orbital elements
elts_pert = perturb_elts(elts_ast, sigma_a=sigma_a, sigma_e=sigma_e, 
                         sigma_inc_deg=sigma_inc_deg, sigma_f_deg=sigma_f_deg, 
                         sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                         mask_pert=mask_pert, random_seed=random_seed)

In [11]:
# Review perturbed elements
# elts_pert

In [12]:
# Random elements
elts_rand = random_elts(element_id_start=0, size=batch_size,
                        random_seed=random_seed, dtype=dtype_np)

In [13]:
# Review random elements
# elts_rand

## Batches of ZTF Data vs. Elements

In [14]:
# Arguments to make_ztf_batch
# thresh_deg = 1.0
# thresh_deg = 2.0
thresh_deg = 4.0
near_ast = False
regenerate = False

In [15]:
# Load unperturbed element batch
ztf_elt_ast = load_ztf_batch(elts=elts_ast, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [16]:
# Load perturbed element batch
ztf_elt_pert = load_ztf_batch(elts=elts_pert, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [17]:
# Load random element batch
ztf_elt_rand = load_ztf_batch(elts=elts_rand, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [18]:
# Score by element - unperturbed
score_by_elt_ast = ztf_score_by_elt(ztf_elt_ast)

In [19]:
# Score by element - perturbed
score_by_elt_pert = ztf_score_by_elt(ztf_elt_pert)

In [20]:
# Score by element - random
score_by_elt_rand = ztf_score_by_elt(ztf_elt_rand)

In [21]:
# Summarize the ztf element batch: unperturbed asteroids
ztf_elt_summary(ztf_elt_ast, score_by_elt_ast, 'Unperturbed Asteroids')

ZTF Element Dataframe Unperturbed Asteroids:
                  Total     (Per Batch)
Observations   :  1233691   (    19276)

Summarize score = sum(-1.0 - log(v)) by batch.  (Mean=0, Variance=num_obs)
Mean score     :    3347.31
Sqrt(batch_obs):     138.84
Mean t_score   :      26.13


In [22]:
# Summarize the ztf element batch: perturbed asteroids
ztf_elt_summary(ztf_elt_pert, score_by_elt_pert, 'Perturbed Asteroids')

ZTF Element Dataframe Perturbed Asteroids:
                  Total     (Per Batch)
Observations   :  1148437   (    17944)

Summarize score = sum(-1.0 - log(v)) by batch.  (Mean=0, Variance=num_obs)
Mean score     :     146.79
Sqrt(batch_obs):     133.96
Mean t_score   :       0.90


In [23]:
# Summarize the ztf element batch: random elements
ztf_elt_summary(ztf_elt_rand, score_by_elt_rand, 'Random Elements')

ZTF Element Dataframe Random Elements:
                  Total     (Per Batch)
Observations   :  1056703   (    16773)

Summarize score = sum(-1.0 - log(v)) by batch.  (Mean=0, Variance=num_obs)
Mean score     :      82.21
Sqrt(batch_obs):     129.51
Mean t_score   :       0.28


In [24]:
# Mixture parameters
num_hits: int = 10
R_deg: float = 1.0

In [25]:
# Add mixture parameters to candidate elements
elts_add_mixture_params(elts=elts_ast, score_by_elt=score_by_elt_ast, num_hits=num_hits, R_deg=R_deg)
elts_add_mixture_params(elts=elts_pert, score_by_elt=score_by_elt_pert, num_hits=num_hits, R_deg=R_deg)
elts_add_mixture_params(elts=elts_rand, score_by_elt=score_by_elt_rand, num_hits=num_hits, R_deg=R_deg)

In [ ]:
elts_ast

In [ ]:
# ********************************************************************************************************************* 
def elts_add_hit_rate(elts: pd.DataFrame, score_by_elt: pd.DataFrame, num_hits: int = 10):
    """
    Populate the hit rate h by guessing each element has num_hits hits
    INPUTS:
        elts: DataFrame with columns a, e, inc, Omega, omega, f, epoch
        score_by_elt: DataFrame generated by ztf_score_by_element in ztf_element.py
        num_hits:     Number of hits to assume for computing the initial guess on hit rate h
    OUTPUTS:
        Updated copy of elts frame
    """
    # Filter elts down to the intersection
    idx = elts.index.intersection(score_by_elt.index)
    elts = elts.loc[idx]
    
    # Update column h
    elts['h'] = num_hits / score_by_elt.num_obs.loc[idx].values

    return elts

# ********************************************************************************************************************* 
def elts_add_R_deg(elts: pd.DataFrame, R_deg: float, dtype=dtype):
    """
    Add two columns to a DataFrame of orbital elements for the mixture parameters h and R.
    INPUTS:
        elts: DataFrame with columns a, e, inc, Omega, omega, f, epoch
        R:    Resolution parameter (Cartesian distance)
    OUTPUTS:
        Modifies elts in place by adding column R
    """
    # Number of asteroids in this batch
    N_ast = elts.shape[0]

    # Convert R from degrees to Cartesian
    R = deg2dist(R_deg)

    # Add column for R
    elts['R'] = np.full(fill_value=R, shape=N_ast, dtype=dtype)

# ********************************************************************************************************************* 
def elts_add_mixture_params(elts: pd.DataFrame, score_by_elt: pd.DataFrame, num_hits: int, R_deg: float, dtype=dtype):
    """
    Add two columns to a DataFrame of orbital elements for the mixture parameters h and R.
    INPUTS:
        elts: DataFrame with columns a, e, inc, Omega, omega, f, epoch
        score_by_elt: DataFrame generated by ztf_score_by_element in ztf_element.py
        num_hits:     Number of hits to assume for computing the initial guess on hit rate h
        R_deg:        Resolution parameter in degrees
    OUTPUTS:
        Modifies elts in place by adding columns h and R
    """
    # Number of asteroids in this batch
    N_ast = elts.shape[0]

    # Convert R from degrees to Cartesian
    R = deg2dist(R_deg)

    # Add columns for h and R
    elts = elts_add_hit_rate(elts=elts, score_by_elt=score_by_elt, num_hits=num_hits)
    elts_add_R_deg(elts=elts, R_deg=R_deg)


In [ ]:
elts = elts_rand
score_by_elt = score_by_elt_rand

elts_add_hit_rate(elts=elts, score_by_elt=score_by_elt, num_hits=num_hits)

In [ ]:
score_by_elt

In [ ]:
elts

In [ ]:
# Review unperturbed elements
elts_ast

In [ ]:
elts_ast.columns

In [ ]:
raise ValueError

## View Example DataFrames and Hits

In [ ]:
# Review ztf_elt DataFrame
ztf_elt_ast

In [ ]:
# Review hits
mask = ztf_elt_ast.is_hit
ztf_elt_ast[mask]

In [ ]:
ztf_elt_ast.columns

In [ ]:
# Alias ztf_elt_ast to ztf_elt
ztf_elt = ztf_elt_ast

In [ ]:
# Build numpy array of times
ts_np = ztf_elt.mjd.values.astype(dtype_np)

# Get observation count per element
row_lengths_np = ztf_elt.element_id.groupby(ztf_elt.element_id).count().values.astype(np.int32)

In [ ]:
# Review results
element_id_best = ast_num_best[0]
mask = (ztf_elt.element_id == element_id_best)
hits_best = np.sum(ztf_elt[mask].is_hit)
hit_rate_best = np.mean(ztf_elt[mask].is_hit)
rows_best = np.sum(mask)
s_sec_min = np.min(ztf_elt[mask].s_sec)
idx = np.argmin(ztf_elt.s)
ztf_id = ztf_elt.ztf_id[idx]
# ztf_elt[mask].iloc[idx:idx+1]
print(f'Best asteroid has element_id = {element_id_best}')
print(f'Hit count: {hits_best} / {rows_best} observations')
print(f'Hit rate : {hit_rate_best:8.6f}')
print(f'Closest hit: {s_sec_min:0.3f} arc seconds')
# ztf_elt[mask]

## Build "Gold" Asteroid Search Model (Reference)

In [ ]:
# Ovservatory for ZTF data is Palomar Mountain
site_name = 'palomar'

In [ ]:
# Training parameters
learning_rate = 2.0**-12
clipnorm = 1.0
regenerate = False

In [ ]:
# "Golden elements": unperturbed, with sharp resolution setting and approximately correct hit rate
elts_gold = elts_ast.copy()
ztf_elt_gold = ztf_elt_ast.copy()
# add_mixture_params(elts_gold, h=h_gold, R_deg=10.0/3600.0, dtype=np.float32)

In [ ]:
# Build asteroid search model
model_gold = AsteroidSearchModel(
        elts=elts_gold, ztf_elt=ztf_elt_gold, 
        site_name=site_name, thresh_deg=thresh_deg, 
        learning_rate=learning_rate, clipnorm=clipnorm)

In [ ]:
# Report summary outputs
model_gold.report()

In [ ]:
# Score using scipy optimization of h, lambda
# elt_score_gold = score_by_elt(ztf_elt_gold, thresh_deg=thresh_deg, fit_mixture=True)
# elt_score_gold = score_by_elt(ztf_elt_gold, thresh_deg=thresh_deg, fit_mixture=False)
# elt_score_gold

In [ ]:
# Freeze orbital elements; train only mixture parameters
model_gold.freeze_candidate_elements()

In [ ]:
# Adaptive search parameters - small size
max_batches = 2000
batches_per_epoch = 100
epochs_per_episode = 5
min_learning_rate = 2.0**-20
verbose = 1

In [ ]:
model_gold.search_adaptive(
    max_batches=max_batches, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=regenerate,
    verbose=verbose)

In [ ]:
model_gold.report()

In [ ]:
# Visualize log likelihood of gold model
model_gold.plot_score_bar('log_like')

## Check that Predicted Direction Matches Expected Direction

In [ ]:
# Predicted direction
u_pred, r_pred = model_gold.predict_direction()

In [ ]:
# Expected direction
cols_u = ['elt_ux', 'elt_uy', 'elt_uz']
u_true = ztf_elt_gold[cols_u].values

In [ ]:
# Difference between actual and predicted
du = u_pred - u_true
u_err = np.linalg.norm(du, axis=-1)
u_err_mean = np.mean(u_err)
u_err_mean_sec = dist2sec(u_err_mean)
print(f'Mean direction error: {u_err_mean:6.2e} Cartesian / {u_err_mean_sec:6.2f} arc seconds')

## Build Asteroid Search Model

In [ ]:
# Review candidate elements
elts_ast

In [ ]:
# elts_ast.h = 1.0/256.0

In [ ]:
# Build asteroid search model
model = AsteroidSearchModel(
        elts=elts_ast, ztf_elt=ztf_elt, 
        site_name=site_name, thresh_deg=thresh_deg, 
        learning_rate=learning_rate, clipnorm=clipnorm)

In [ ]:
# Report before training starts
model.report()

In [ ]:
# Built in log likelihood caclulation
# model.calc_log_like()

In [ ]:
# Visualize model summary - layers and parameters
model.summary()

In [ ]:
# Visualize log likelihood before traning
model.plot_score_bar('log_like', sorted=False)

## Fit Model on Unperturbed Elements

In [ ]:
# Training parameters
learning_rate = 2.0**-12
clipnorm = 1.0
regenerate = False

In [ ]:
# Build asteroid search model
model_ast = AsteroidSearchModel(
                elts=elts_ast, ztf_elt=ztf_elt, 
                site_name=site_name, thresh_deg=thresh_deg, 
                learning_rate=learning_rate, clipnorm=clipnorm)

In [ ]:
# Report before training starts
model.report()

In [ ]:
# Adaptive search parameters
max_batches_mixture = 10000
max_batches_element = 20000
batches_per_epoch = 10
epochs_per_episode = 5
max_bad_episodes = 3
learning_rate = 2.0**-12
min_learning_rate = None
verbose = 1

In [ ]:
# # Tiny size for fast testing
# max_batches = 1000
# batches_per_epoch = 10

In [ ]:
# Preliminary round of training with frozen elements
model_ast.freeze_candidate_elements()

In [ ]:
# Train unperturbed model with frozen orbital elements
model_ast.search_adaptive(
    max_batches=max_batches_mixture, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=True,
    verbose=verbose)

In [ ]:
# Unfreeze the elements, freeze the mixture model parameters
model_ast.thaw_candidate_elements()
model_ast.freeze_mixture_parameters()

In [ ]:
# Train unperturbed model with frozen orbital elements
model_ast.search_adaptive(
    max_batches=max_batches_element, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=regenerate,
    verbose=verbose)

In [ ]:
# Report after training
model_ast.report()

In [ ]:
model_gold.plot_score_bar('log_like', episode=0)

In [ ]:
# Bar chart: log likelihood
model_ast.plot_score_bar('log_like', episode=0)

In [ ]:
# Bar chart: hits
model_ast.plot_score_bar('hits')

In [ ]:
# Learning curve: log likelihood
model_ast.plot_score_hist('log_like')

In [ ]:
# Learning curve: hits
model_ast.plot_score_hist('hits')

In [ ]:
# Mixture parameter: resolution
model_ast.plot_mixture_param('R_deg')

In [ ]:
# Plot error in orbital elements
model_ast.plot_elt_error(elts_true=elts_ast, elt_name='a', is_log=True, elt_num=None)

In [ ]:
# Plot error in orbital elements
model_ast.plot_elt_error(elts_true=elts_ast, elt_name='e', is_log=True, elt_num=None)

In [ ]:
# Plot control variables for worst element
model_ast.plot_control(element_num=55)

In [ ]:
# Plot control variables for best element
model_ast.plot_control(element_num=59)

## Train on Perturbed Elements

In [ ]:
# Build asteroid search model
model_pert = AsteroidSearchModel(
                 elts=elts_pert, ztf_elt=ztf_elt_pert, site_name=site_name,
                 thresh_deg=thresh_deg, h=h, R_deg=R_deg,
                 learning_rate=learning_rate, clipnorm=clipnorm)

In [ ]:
# Freeze elements
model_pert.freeze_elements()

In [ ]:
# Train model on perturbed elements
model_pert.search_adaptive(
    max_batches=max_batches_mixture, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=True,
    verbose=verbose)

In [ ]:
# Unfreeze the elements, freeze the mixture model parameters
model_pert.thaw_candidate_elements()
model_pert.freeze_mixture_parameters()

In [ ]:
# Train unperturbed model with frozen orbital elements
model_pert.search_adaptive(
    max_batches=max_batches_element, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=True,
    verbose=verbose)

In [ ]:
model_pert.report()

## Train on Random Elements

In [ ]:
# Filter elts_rand down to only those that had matching ztf observations
idx = np.unique(ztf_elt_rand.element_id)
elts_rand = elts_rand.loc[idx]

In [ ]:
elts_rand['a'].dtype

In [ ]:
elts_rand['a'].dtype

In [ ]:
for col in ['a', 'e', 'f', 'inc', 'Omega', 'omega', 'epoch']:
    elts_rand[col] = elts_rand[col].astype(np.float32)

In [ ]:
# Build asteroid search model
model_rand = AsteroidSearchModel(
                 elts=elts_rand, ztf_elt=ztf_elt_rand, site_name=site_name,
                 thresh_deg=thresh_deg, h=h, R_deg=R_deg,
                 learning_rate=learning_rate, clipnorm=clipnorm)

In [ ]:
# Train model on perturbed elements
model_rand.search_adaptive(
    max_batches=max_batches, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    verbose=verbose)

In [ ]:
# Review likelihoods by element
log_like_rand, orbital_elements_rand, mixture_parameters_rand = model_rand.calc_outputs()

In [ ]:
log_like_rand

In [ ]:
# orbital_elements_rand